In [5]:
from environments.quadrotor import QuadrotorEnv
from environments.config.config_loader import trpo_ppo_default_params
import yaml
import gym
import numpy as np

In [2]:
env = gym.make('Quadrotor-v1')

QuadEnv: Dyn update time:  0.03356671333312988


/home/naliseas-workstation/anaconda3/envs/haitong_botorch/lib/python3.9/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/naliseas-workstation/anaconda3/envs/haitong_botorch/lib/python3.9/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/naliseas-workstation/anaconda3/envs/haitong_botorch/lib/python3.9/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [12]:
default_params = trpo_ppo_default_params()
params_path = '/home/naliseas-workstation/Documents/haitong/sim_to_real/quad_sim/environments/config/ppo__crazyflie_baseline_randomize_t2w15_35.yml'
yaml_stream = open(params_path, 'r')
params = yaml.load(yaml_stream, Loader=yaml.Loader)
params['variant']["env_param"]

{'dynamics_params': 'crazyflie_t2w_15_35',
 'dynamics_randomize_every': 1,
 'raw_control': True,
 'tf_control': False,
 'dim_mode': '3D',
 'init_random_state': True,
 'sim_freq': 200,
 'sim_steps': 2,
 'ep_time': 7,
 'sense_noise': 'default',
 'rew_coeff': {'pos': 1.0,
  'pos_log_weight': 0.0,
  'pos_linear_weight': 1.0,
  'effort': 0.05,
  'spin': 0.1,
  'vel': 0.0,
  'crash': 0.0,
  'orient': 0.0,
  'yaw': 0.0}}

In [10]:
env.dim_mode

'3D'

In [24]:
state = env.step(env.action_space.sample())[0]

In [26]:
buffer = np.zeros([100, 18])
buffer[0:8] = state

In [2]:
import torch
from train.agent.sac.actor import DiagGaussianActor


def get_firmware_file(log_path):

    actor = DiagGaussianActor(obs_dim=18,
                              action_dim=4,
                              hidden_dim=64,
                              hidden_depth=2,
                              log_std_bounds=[-5., 2.]) # hard coded for drone controllers.
    # if use_nystrom == False:
    #     critic = RFVCritic(**kwargs)
    # else:
    #     critic = nystromVCritic(**kwargs)

    actor.load_state_dict(torch.load(log_path))
    return actor

In [3]:
actor = get_firmware_file('train/log/Quadrotor-v1/sac/reproduce_speder/1/best_actor.pth')

In [45]:
actor.parameters()

<generator object Module.parameters at 0x7fd29bda3ac0>

In [5]:
w0 = actor.trunk[0].weight.detach().numpy()
b0 = actor.trunk[0].bias.detach().numpy()
w1 = actor.trunk[2].weight.detach().numpy()
b1 = actor.trunk[2].bias.detach().numpy()
w2 = actor.trunk[4].weight.detach().numpy()
b2 = actor.trunk[4].bias.detach().numpy()

In [8]:
env = QuadrotorEnv()
state = env.reset()

QuadEnv: Dyn update time:  0.00185394287109375


/home/mht/anaconda3/envs/gpyt/lib/python3.8/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [37]:
dist = actor(torch.from_numpy(state).float())
dist.mean

tensor([-0.8238,  0.0026, -0.0215, -0.5814], grad_fn=<TanhBackward0>)

In [19]:
def elu(x):
    elu_x = []
    for xi in x:
        if xi >= 0:
            elu_x.append(xi)
        else:
            elu_x.append(np.exp(xi) - 1)
    return np.array(elu_x)

In [40]:
x = np.matmul(w0, state.astype(float)) + b0
x = elu(x)
x = np.matmul(w1, x) + b1
x = elu(x)
x = np.matmul(w2, x) + b2
np.tanh(x)

array([-0.82384596,  0.0025772 , -0.02153534, -0.58137425,  0.22800406,
        0.23380311,  0.25142225,  0.25680471])

array([-0.82384596,  0.0025772 , -0.02153534, -0.58137425,  0.22800406,
        0.23380311,  0.25142225,  0.25680471])

In [27]:
m = actor.trunk[1]

In [32]:
m(torch.tensor(-1.))

tensor(-0.6321)